# Loop 6 Analysis: Double Lattice Packing Investigation

The web search revealed that top teams use **dense 2-tree lattices with alternating up/down oriented trees** and **180° rotational symmetry**.

Let's analyze:
1. What patterns are in our current best solutions?
2. Are we using the optimal double-lattice pattern?
3. Where can we improve?

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely.ops import unary_union
import matplotlib.pyplot as plt

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

def get_tree_polygon(cx, cy, angle_deg):
    angle_rad = np.radians(angle_deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    x = TX * cos_a - TY * sin_a + cx
    y = TX * sin_a + TY * cos_a + cy
    return Polygon(zip(x, y))

def get_bounding_box_side(trees):
    all_poly = unary_union(trees)
    bounds = all_poly.bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

# Load our current best solution
df = pd.read_csv('/home/submission/submission.csv')
for col in ['x', 'y', 'deg']:
    if df[col].dtype == object:
        df[col] = df[col].str.strip('s').astype(float)
df['n'] = df['id'].str[:3].astype(int)

print(f"Loaded {len(df)} rows")
print(df.head())

In [ ]:
# Analyze angle distribution for each N
print("Angle distribution analysis:")
print("="*60)

for n in [2, 4, 8, 10, 20, 50, 100, 200]:
    group = df[df['n'] == n]
    angles = group['deg'].values
    
    # Normalize angles to 0-360
    angles_norm = angles % 360
    
    # Count unique angles
    unique_angles = np.unique(np.round(angles_norm, 1))
    
    # Check for 180-degree symmetry (up/down pattern)
    has_0 = np.any(np.abs(angles_norm) < 5) or np.any(np.abs(angles_norm - 360) < 5)
    has_180 = np.any(np.abs(angles_norm - 180) < 5)
    
    print(f"\nN={n}:")
    print(f"  Unique angles: {unique_angles[:10]}..." if len(unique_angles) > 10 else f"  Unique angles: {unique_angles}")
    print(f"  Has 0°: {has_0}, Has 180°: {has_180}")
    print(f"  Up/Down pattern: {has_0 and has_180}")
    
    # Count trees at each angle
    angle_counts = {}
    for a in angles_norm:
        key = round(a / 45) * 45  # Round to nearest 45 degrees
        angle_counts[key] = angle_counts.get(key, 0) + 1
    print(f"  Angle counts (rounded to 45°): {angle_counts}")

In [ ]:
# Visualize solutions for key N values
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for idx, n in enumerate([2, 4, 8, 14, 20, 50, 100, 200]):
    ax = axes[idx // 4, idx % 4]
    group = df[df['n'] == n]
    
    for _, row in group.iterrows():
        poly = get_tree_polygon(row['x'], row['y'], row['deg'])
        x, y = poly.exterior.xy
        
        # Color by angle: 0° = blue, 180° = red, other = green
        angle = row['deg'] % 360
        if abs(angle) < 10 or abs(angle - 360) < 10:
            color = 'blue'
        elif abs(angle - 180) < 10:
            color = 'red'
        else:
            color = 'green'
        
        ax.fill(x, y, alpha=0.3, color=color)
        ax.plot(x, y, color=color, linewidth=0.5)
    
    ax.set_aspect('equal')
    ax.set_title(f'N={n}')

plt.tight_layout()
plt.savefig('/home/code/exploration/current_solutions_viz.png', dpi=100)
plt.show()
print("Blue = 0°, Red = 180°, Green = other angles")

In [ ]:
# Calculate theoretical optimal using double-lattice pattern
# Based on the zaburo kernel approach

def calculate_double_lattice_score(n):
    """Calculate score using alternating up/down pattern."""
    best_score = float('inf')
    
    # Try different row configurations
    for n_even in range(1, n + 1):
        for n_odd in [n_even, n_even - 1, n_even + 1]:
            if n_odd < 0:
                continue
            
            all_trees = []
            rest = n
            r = 0
            
            while rest > 0:
                m = min(rest, n_even if r % 2 == 0 else n_odd)
                if m <= 0:
                    break
                rest -= m
                
                angle = 0 if r % 2 == 0 else 180
                x_offset = 0 if r % 2 == 0 else 0.35  # Half of 0.7
                y = r // 2 * 1.0 if r % 2 == 0 else (0.8 + (r - 1) // 2 * 1.0)
                
                for i in range(m):
                    cx = 0.7 * i + x_offset
                    cy = y
                    all_trees.append(get_tree_polygon(cx, cy, angle))
                
                r += 1
            
            if len(all_trees) == n:
                side = get_bounding_box_side(all_trees)
                score = side ** 2 / n
                if score < best_score:
                    best_score = score
    
    return best_score

# Compare with our current scores
print("Comparing double-lattice pattern with current solutions:")
print("="*60)

for n in [2, 4, 8, 10, 14, 20, 50, 100]:
    group = df[df['n'] == n]
    trees = [get_tree_polygon(row['x'], row['y'], row['deg']) for _, row in group.iterrows()]
    current_side = get_bounding_box_side(trees)
    current_score = current_side ** 2 / n
    
    lattice_score = calculate_double_lattice_score(n)
    
    diff = current_score - lattice_score
    print(f"N={n:3d}: Current={current_score:.6f}, Lattice={lattice_score:.6f}, Diff={diff:+.6f}")

In [ ]:
# Calculate total score comparison
print("\nTotal score comparison:")
print("="*60)

current_total = 0
lattice_total = 0

for n in range(1, 201):
    group = df[df['n'] == n]
    if len(group) != n:
        print(f"Warning: N={n} has {len(group)} trees")
        continue
    
    trees = [get_tree_polygon(row['x'], row['y'], row['deg']) for _, row in group.iterrows()]
    current_side = get_bounding_box_side(trees)
    current_score = current_side ** 2 / n
    current_total += current_score
    
    # Only calculate lattice for larger N (small N are already optimal)
    if n >= 10:
        lattice_score = calculate_double_lattice_score(n)
        lattice_total += lattice_score
    else:
        lattice_total += current_score

print(f"Current total: {current_total:.6f}")
print(f"Lattice total: {lattice_total:.6f}")
print(f"Difference: {current_total - lattice_total:+.6f}")
print(f"\nTarget: 68.919154")
print(f"Gap from current: {current_total - 68.919154:.6f}")

In [ ]:
# Analyze which N values have the most room for improvement
print("\nPer-N score analysis (sorted by contribution to total):")
print("="*70)

scores = []
for n in range(1, 201):
    group = df[df['n'] == n]
    if len(group) != n:
        continue
    trees = [get_tree_polygon(row['x'], row['y'], row['deg']) for _, row in group.iterrows()]
    side = get_bounding_box_side(trees)
    score = side ** 2 / n
    scores.append((n, side, score))

# Sort by score contribution
scores.sort(key=lambda x: -x[2])

print(f"{'N':>4} {'Side':>10} {'Score':>10} {'Cumulative':>12}")
print("-" * 40)
cumulative = 0
for n, side, score in scores[:30]:
    cumulative += score
    print(f"{n:4d} {side:10.6f} {score:10.6f} {cumulative:12.6f}")

print(f"\nTop 30 N values contribute {cumulative:.2f} out of {current_total:.2f} ({100*cumulative/current_total:.1f}%)")

In [ ]:
# Check if our solutions for N=2, 4, 8, 14 match known optimal patterns
print("\nVerifying known optimal solutions:")
print("="*60)

# N=1: Single tree at 45 degrees is optimal
n = 1
group = df[df['n'] == n]
angle = group['deg'].values[0] % 360
print(f"N=1: angle={angle:.1f}° (optimal is 45°)")

# N=2: Two trees should be at 0° and 180° (or similar symmetric)
n = 2
group = df[df['n'] == n]
angles = sorted(group['deg'].values % 360)
print(f"N=2: angles={angles} (optimal is symmetric up/down)")

# N=4: Should form a 2x2 grid with alternating orientations
n = 4
group = df[df['n'] == n]
angles = sorted(group['deg'].values % 360)
print(f"N=4: angles={angles}")

# N=8: Should be 4x2 or similar with alternating
n = 8
group = df[df['n'] == n]
angles = sorted(group['deg'].values % 360)
print(f"N=8: angles={angles}")

# N=14: Known optimal
n = 14
group = df[df['n'] == n]
angles = sorted(group['deg'].values % 360)
print(f"N=14: angles={angles[:5]}... (showing first 5)")